In [ ]:
from random import seed
from csv import reader
import os 
import sys
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import jaccard_similarity_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_mldata
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split 
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity
from imblearn.under_sampling import CondensedNearestNeighbour, RepeatedEditedNearestNeighbours, EditedNearestNeighbours
import random
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE 
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from scipy import stats



#Read data

In [ ]:
# Size of the data frame
m,n=data.shape

#Select train and labels
X = data.iloc[:,1:n]
y = data.iloc[:,0]
y = np.array(y)
X = np.array(X)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state= 0)


df_mean = list()
act = np.unique(y)
len_act = len(act)

for i in act:
    A = y_train == i
    d = X_train[A]
    temp = "df"+str(i) 
    vars()[temp] = d.reset_index(drop=True)
    r,c = vars()[temp].shape
    X0 = vars()[temp].iloc[:,0:c]
    
    db_cluster = DBSCAN(eps=.25, min_samples=10).fit(X0)
    
    X0_aux = pd.DataFrame(X0)
    X0_aux['label'] = pd.DataFrame(db_cluster.labels_)
    
    lab = np.unique(db_cluster.labels_)
    
    for l in lab:
        mean_aux = X0_aux[X0_aux['label']==l]
        mean_cl = np.mean(mean_aux.iloc[:,0:mean_aux.shape[1]-1])
        df_mean.append((np.asarray(mean_cl)))

df_cl = pd.DataFrame(np.array(df_mean).T)        

# Size of the data frame

#Xt = data_aux.iloc[:,0:11]
m,n = X_train.shape
Xt = np.array(X_train)
ps = np.array(df_cl).T
#ps = np.array(clusters)
cm,cn = ps.shape

dist = np.empty((m,cm))

for w in range(m):
    d1 = Xt[w].reshape(1, -1)
    for t in range(cm):
        dist[w][t] = pairwise_distances(d1,ps[t].reshape(1,-1),'cosine') 

#Xt = data_aux.iloc[:,0:11]
m,n = X_test.shape
Xt = np.array(X_test)
ps = np.array(df_cl).T
#ps = np.array(clusters)
cm,cn = ps.shape
dist_test = np.empty((m,cm))

for w in range(m):
    d1 = Xt[w].reshape(1, -1)
    for t in range(cm):
        dist_test[w][t] = pairwise_distances(d1,ps[t].reshape(1,-1),'cosine') 

        
parameters  = {'C':[.1,1],'gamma':[.1,1]}
#smv_dism_data1 = svm.SVC(kernel='rbf', probability = True)
smv_dism_data1 = svm.SVC(kernel='rbf', probability = True, class_weight = 'balanced')
clf = GridSearchCV(smv_dism_data1, parameters)
predicted_data1 = clf.fit(X_train, y_train)
#predicted_data1 = smv_dism_data1.fit(X_train, y_train)
pred_proba_test = predicted_data1.predict_proba(dist_test)
pred_label = predicted_data1.predict(dist_test)
rs = recall_score(y_test, pred_label, average='macro') 
ps = precision_score(y_test, pred_label, average='macro') 


#### Filter misclassified instances
distc=X_test.copy()
distc = pd.DataFrame(distc)
distc['label'] = pd.DataFrame(y_test)
distc['SVM_label'] = pd.DataFrame(pred_label) 
distc['label_b'] = np.where((distc['label'] == distc['SVM_label']), 1, 0)
A = distc['label_b'] == 0 #misclassified
d = distc[A]
c_class = distc[-A]
distc_new = d.reset_index(drop=True)
df_cc = c_class.reset_index(drop=True)
label_df = distc_new[['label','SVM_label']] 

x_cnn = distc_new.iloc[:,0:distc_new.shape[1]-3]
y_cnn = distc_new.iloc[:,distc_new.shape[1]-3]
cnn = CondensedNearestNeighbour(return_indices=True)
X_resampled, y_resampled, idx_resampled = cnn.fit_sample(x_cnn, y_cnn)
Xcnn = pd.DataFrame(X_resampled)
Xcnn['label'] = pd.DataFrame(y_resampled)
x_cnn['label'] = y_cnn
Xcnn = Xcnn.append(x_cnn)
Xsplit = Xcnn.iloc[:,0:Xcnn.shape[1]]
ysplit = Xcnn.iloc[:, Xcnn.shape[1]-1]
X_train_m, X_test_m, y_train_m, y_test_ = train_test_split(Xsplit,ysplit, test_size=0.2, random_state= 0)

#### Create subspaces
X_train_m = pd.DataFrame(X_train_m)
dist_cl = X_train_m.iloc[:,0:X_train_m.shape[1]-1]
db_cluster_sub = DBSCAN(eps= .25, min_samples=10).fit(dist_cl)   
X_train_m['cl_label'] = pd.DataFrame(db_cluster_sub.labels_)
X_train_m['cl_label'] = np.nan_to_num(np.array(X_train_m['cl_label']))  

### Filter subspaces
lab_aux =  np.unique(np.array(X_train_m['cl_label']))
subspace = list()
for lab in lab_aux:
    A_aux = X_train_m['cl_label'] == lab
    daux = X_train_m[A_aux]
    daux = daux.reset_index(drop=True)
    if daux.shape[0]>2:
        subspace.append(daux)
        
        
rdf = c_class.iloc[:,0:c_class.shape[1]-2]

for s in range(len(subspace)):
    dfs = subspace[s]
    count_class = np.unique(np.array(dfs['label']),return_counts=True)
    num_class = count_class[1]
    for n in range(len(num_class)):
        if num_class[n] < 10:
            
            class_s = []
            class_s.append(count_class[0][n])
            svmlab = label_df['label'] == class_s[0]
            svm_lab = label_df[svmlab]
            false_label = np.unique(svm_lab['SVM_label'])
            class_s = np.concatenate((false_label,class_s),axis=0)
            

            for cf in range(len(class_s)):
                tc_df = rdf['label'] == class_s[cf]
                tcdfaux = rdf[tc_df]
                tcdfaux = tcdfaux.reset_index(drop=True)
                if tcdfaux.shape[0] >0 :
                    subspace[s] = subspace[s].append(tcdfaux.sample(num_class[n]+6,replace=True))
                else:
                    tc = pd.DataFrame(X_train)
                    tc['label'] = pd.DataFrame(y_train)
                    tc_df = tc['label'] == class_s[cf]
                    tcdfaux = tc[tc_df]
                    tcdfaux = tcdfaux.reset_index(drop=True)
                    subspace[s] = subspace[s].append(tcdfaux.sample(num_class[n]+6,replace=True))


#Resample with SMOTE Technique + CNN

new_subspace = list()
for s in range(len(subspace)):
    dfs = subspace[s].iloc[:,0:subspace[s].shape[1]-1]
    
    dfs_x = dfs.iloc[:,0:dfs.shape[1]-1]
    dfs_y = dfs.iloc[:, dfs.shape[1]-1]
    
    sm = SMOTE(ratio = 'auto', random_state=0)
    X_res, y_res = sm.fit_sample(dfs_x, dfs_y)
    
    X_resv = pd.DataFrame(X_res)
    X_resv['y_r'] = pd.DataFrame(y_res)
    
    #new_subspace.append(X_res)
    
    # Apply Condensed Nearest Neighbours
    cnn = CondensedNearestNeighbour(return_indices=True)
    X_resampled, y_resampled, idx_resampled = cnn.fit_sample(X_res, y_res)

    #Select train and labels
    X_resampled = pd.DataFrame(X_resampled)
    X_resampled['y_r'] = pd.DataFrame(y_resampled)
    X_resv = X_resv.append(X_resampled, ignore_index=True)
    new_subspace.append(X_resv)
    

len_sub = len(new_subspace)
proba_dist = list()
true_test = list()
pred_test = list()
proba_dist = list()
predlabel = list()
classes = list()
test_df = pd.DataFrame(X_test_m.iloc[:,0:X_test_m.shape[1]-1])
new_sub = list()


for l in range(len(new_subspace)):
    
    df0 = new_subspace[l]
    X_aux = df0.iloc[:,0:df0.shape[1]-1]
    y_aux = df0.iloc[:,df0.shape[1]-1]
       
    #SVM   
    parameters  = {'C':[.1,1],'gamma':[.1,1]}
    #smv_sub = svm.SVC(kernel='rbf', probability = True)
    smv_sub = svm.SVC(kernel='rbf', probability = True, class_weight = 'balanced')
    clf = GridSearchCV(smv_sub, parameters)
    predicted_sub = clf.fit(X_aux, y_aux)
    pred_prob_sub = predicted_sub.predict_proba(test_df)
    #Save probability distribution
    proba_dist.append((pred_prob_sub))
    predlabel.append((predicted_sub.predict(test_df)))
    #Classes
    classes.append((predicted_sub.classes_))

    
#Ensemble part
len_p = len(proba_dist)
ensemble = np.zeros((test_df.shape[0],len_p),dtype=int)

for num in range(len_p):
    class_vec = classes[num]
    for row in range(proba_dist[num].shape[0]):
        row_max = proba_dist[num][row].max()
        id_max = np.argmax(proba_dist[num][row])
        ensemble[row][num] = class_vec[id_max]

ensemble_pred = np.zeros(test_df.shape[0],dtype=int)

for row in range(ensemble.shape[0]):
    ensemble_pred[row] = stats.mode(ensemble[row])[0][0]
    
